## ml-100k \ text \ items.csv -----> baby \ text_feat.npy


In [ ]:
import csv
import torch
import numpy as np

file_path = 'items.csv'


with open(file_path, 'r') as file:
    data_reader = csv.reader(file)
    data = [row for row in data_reader]

# for i in range(len(data[0])):
#     print("i: ", i)
#     print("data[0][i]: ", data[0][i])
ind1 = [5, 29, 30, 31, 32]
# movie title-5, summary-28, cast-29, director-30, rating-31, runtime-32

moviedesc = []
for i in range(1, len(data)):
    tmp=""
    for j in ind1:
        tmp += data[i][j]
        tmp += " "
    # genres
    for j in range(10, 28):
        if(data[i][j]=='1'):
            tmp += data[0][j]
            tmp += " "
    moviedesc.append(tmp)

print("len(moviedesc): ", len(moviedesc))
print(moviedesc)

# making embeddings using huggingface library

from transformers import AutoTokenizer, CLIPTextModelWithProjection

model = CLIPTextModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

inputs = tokenizer(moviedesc, padding=True, return_tensors="pt")

outputs = model(**inputs)
text_embeds = outputs.text_embeds
# print(text_embeds.size())
print("len(text_embeds): ", len(text_embeds))
resized_text_embeds = []


# resizing text embeddings to 384 
for t in text_embeds:
    resized_t = torch.nn.functional.interpolate(t.unsqueeze(0).unsqueeze(0), size = (384,), mode = 'linear').squeeze()
    resized_text_embeds.append(resized_t)

print("len(resized_text_embeds): ", len(resized_text_embeds))
# saving text embeddings in text_feat.npy
numpy_array = [tensor.detach().numpy() for tensor in resized_text_embeds]
print("len(numpy_array): ", len(numpy_array))
stacked_resized_text_embeds = np.stack(numpy_array, axis=0)
print("stacked_resized_text_embeds.shape: ", stacked_resized_text_embeds.shape)
np.save('text_feat.npy',stacked_resized_text_embeds)
print("saved text embeddings in text_feat.npy")


## extracting middle frame from video

In [ ]:
import cv2
import os

def extract_middle_frame(video_path, output_folder):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Check if the video opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video '{video_path}'")
        return

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the middle frame index
    middle_frame_index = total_frames // 2

    # Set the video capture object to the middle frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame_index)

    # Read the middle frame
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        print(f"Error: Could not read middle frame from video '{video_path}'")
        return

    # Close the video file
    cap.release()

    # Save the middle frame as an image
    frame_filename = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(video_path))[0]}_middle_frame.jpg")
    cv2.imwrite(frame_filename, frame)
    print(f"Saved the middle frame of '{video_path}' as '{frame_filename}'")

# Specify the path to your video dataset folder
video_dataset_folder = "C:/Users/richi/OneDrive/Documents/COL865_MMRS/videos"

# Specify the folder where you want to save the extracted frames
output_image_folder = "C:/Users/richi/OneDrive/Documents/COL865_MMRS/images"

# Loop through the video files in the dataset folder
for video_filename in os.listdir(video_dataset_folder):
    video_path = os.path.join(video_dataset_folder, video_filename)
    extract_middle_frame(video_path, output_image_folder)

## converting extracted images into embeddings

In [ ]:
import os
import librosa

from PIL import Image
# import requests
from transformers import AutoProcessor, CLIPVisionModelWithProjection

jpg_dir = "C:/Users/richi/OneDrive/Documents/COL865_MMRS/images"


model = CLIPVisionModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

for filename in os.listdir(jpg_dir):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
        file_path = os.path.join(jpg_dir, filename)
        image_sample =Image.open(file_path)
        inputs = processor(images=image_sample, return_tensors="pt")
        outputs = model(**inputs)
        image_embeds = outputs.image_embeds
        print(image_embeds)

In [ ]:
import os
import librosa
import csv
import torch
import numpy as np

from PIL import Image
# import requests
from transformers import AutoProcessor, CLIPVisionModelWithProjection

# jpg_dir =  "/Users/manshisagar/Desktop/sem-7 assign/col865/project/COL865_MMRS/ml-100k/Image"
# ekansh
jpg_dir = "/Users/manshisagar/Desktop/sem-7assign/col865/project/COL865_MMRS/movie_posters_resized"

model = CLIPVisionModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

# def resize_and_preprocess_image(image_path, target_size=(256, 256)):
#     image = Image.open(image_path)
#     image = image.resize(target_size)
#     inputs = processor(images=image, return_tensors="pt")
#     return inputs

image_files = [filename for filename in os.listdir(jpg_dir) if filename.lower().endswith(('.jpg', '.png', '.jpeg'))]
print(len(image_files))



arr_image_embeds= []

# Loop through all the image files and process them
for i in range(len(image_files)):
    # print(filename)
    filename = image_files[i]
    print(i)
    file_path = os.path.join(jpg_dir, filename)
    image_sample = Image.open(file_path)
    inputs = processor(images=image_sample, return_tensors="pt")
    outputs = model(**inputs)
    image_embeds = outputs.image_embeds
    arr_image_embeds.append(image_embeds)

# for i in range(len(image_files)):
#     filename = image_files[i]
#     print(i)
#     file_path = os.path.join(jpg_dir, filename)
#     inputs = resize_and_preprocess_image(file_path)
#     outputs = model(**inputs)
#     image_embeds = outputs.image_embeds
#     arr_image_embeds.append(image_embeds)

print("size of tensor == ", len(arr_image_embeds[0]))
print("size of arr_image_embeds== ", len(arr_image_embeds))
new_image_embeds= []
for t in arr_image_embeds:
    temp= t.squeeze()
    new_image_embeds.append(temp)

print("size of tensor == ", len(new_image_embeds[0]))
print("size of new_image_embeds== ", len(new_image_embeds))
resized_image_embeds= []
# resizing text embeddings to 384 
for t in new_image_embeds:
    resized_i = torch.nn.functional.interpolate(t.unsqueeze(0).unsqueeze(0), size = (4096,), mode = 'linear').squeeze()
    resized_image_embeds.append(resized_i)

print("size of new tensor == ", resized_image_embeds[0].size())
print("size of resized_image_embeds== ", len(resized_image_embeds))
# saving text embeddings in text_feat.npy
numpy_array = [tensor.detach().numpy() for tensor in resized_image_embeds]

stacked_resized_text_embeds = np.stack(numpy_array, axis=0)

np.save('images_feat.npy',stacked_resized_text_embeds)
print("saved text embeddings in image_feat.npy")